# Deployment Of Customer Churn Prediction Model 


### Create a python function


In [4]:
# This Function can be called from any from any front end tool/website
def FunctionPredictResult(InputData):
    import pandas as pd
    Num_Inputs=InputData.shape[0]
    
    # Making sure the input data has same columns as it was used for training the model
    # Also, if standardization/normalization was done, then same must be done for new input
    
    # Appending the new data with the Training data
    DataForML=pd.read_pickle('C:/Users/anann/E-commerce Customer Churn Analysis and Prediction/FinalChurnData.pkl')
    InputData=InputData.append(DataForML)
            
    # Maintaining the same order of columns as it was during the model training
    Predictors=['Tenure', 'WarehouseToHome', 'CashbackAmount', 
       'NumberOfDeviceRegistered', 'SatisfactionScore', 'NumberOfAddress',
       'Complain', 'OrderAmountHikeFromlastYear', 'OrderCount',
       'DaySinceLastOrder']
    
    # Generating the input values to the model
    X=InputData[Predictors].values[0:Num_Inputs]
    
    ### Sandardization of data ###
    from sklearn.preprocessing import StandardScaler, MinMaxScaler

    PredictorScaler=MinMaxScaler()

    # Storing the fit object for later reference
    PredictorScalerFit=PredictorScaler.fit(X)

    # Generating the standardized values of X since it was done while model training also
    X=PredictorScalerFit.transform(X)
    
    # Loading the Function from pickle file
    import pickle
    with open('C:/Users/anann/E-commerce Customer Churn Analysis and Prediction/FinalRFModel.pkl', 'rb') as fileReadStream:
        PredictionModel=pickle.load(fileReadStream)
        # Don't forget to close the filestream!
        fileReadStream.close()
            
    # Genrating Predictions
    Prediction=PredictionModel.predict(X)
    PredictionResult=pd.DataFrame(Prediction, columns=['Predicted Status'])
    return(PredictionResult)

In [14]:
import pandas as pd
# Calling the function for some loan applications
NewSampleData=pd.DataFrame(
data=[[4.0,6,160, 3, 2, 9, 1, 11, 1, 5],
     [12.0,10,200, 3, 4, 3, 0, 20, 5, 10]],
columns=['Tenure', 'WarehouseToHome', 'CashbackAmount', 
       'NumberOfDeviceRegistered', 'SatisfactionScore', 'NumberOfAddress',
       'Complain', 'OrderAmountHikeFromlastYear', 'OrderCount',
       'DaySinceLastOrder'])

print(NewSampleData)

# Calling the Function for prediction
FunctionPredictResult(InputData= NewSampleData)

   Tenure  WarehouseToHome  CashbackAmount  NumberOfDeviceRegistered  \
0     4.0                6             160                         3   
1    12.0               10             200                         3   

   SatisfactionScore  NumberOfAddress  Complain  OrderAmountHikeFromlastYear  \
0                  2                9         1                           11   
1                  4                3         0                           20   

   OrderCount  DaySinceLastOrder  
0           1                  5  
1           5                 10  


,Predicted Status
0,0
1,0


The Function FunctionPredictResult can be used to produce the predictions for one or more cases at a time. Hence, it can be scheduled using a batch job or cron job to run every night and generate predictions for all the available cases.

## Function for predictions API

In [15]:
# Creating the function which can take inputs and return predictions
def FunctionGeneratePrediction(Inp_Tenure, Inp_WarehouseToHome, Inp_CashbackAmount, 
       Inp_NumberOfDeviceRegistered, Inp_SatisfactionScore, Inp_NumberOfAddress,
       Inp_Complain, Inp_OrderAmountHikeFromlastYear, Inp_OrderCount,
       Inp_DaySinceLastOrder):
    
    # Creating a data frame for the model input
    SampleInputData=pd.DataFrame(
     data=[[Inp_Tenure, Inp_WarehouseToHome, Inp_CashbackAmount, 
       Inp_NumberOfDeviceRegistered, Inp_SatisfactionScore, Inp_NumberOfAddress,
       Inp_Complain, Inp_OrderAmountHikeFromlastYear, Inp_OrderCount,
       Inp_DaySinceLastOrder]],
     columns=['Tenure', 'WarehouseToHome', 'CashbackAmount', 
       'NumberOfDeviceRegistered', 'SatisfactionScore', 'NumberOfAddress',
       'Complain', 'OrderAmountHikeFromlastYear', 'OrderCount',
       'DaySinceLastOrder'])

    # Calling the function defined above using the input parameters
    Predictions=FunctionPredictResult(InputData= SampleInputData)

    # Returning the predictions
    return(Predictions.to_json())

# Function call
FunctionGeneratePrediction(  Inp_Tenure=12, 
                             Inp_WarehouseToHome=10, 
                             Inp_CashbackAmount=200, 
                             Inp_NumberOfDeviceRegistered=3, 
                             Inp_SatisfactionScore=4, 
                             Inp_NumberOfAddress=4,
                             Inp_Complain=0, 
                             Inp_OrderAmountHikeFromlastYear=20, 
                             Inp_OrderCount=3,
                             Inp_DaySinceLastOrder=6
                             )

'{"Predicted Status":{"0":0}}'

## Creating Flask API

In [16]:
from flask import Flask, request, jsonify
import pickle
import pandas as pd
import numpy

In [18]:
app = Flask(__name__)

@app.route('/prediction_api', methods=["GET"])
def prediction_api():
    try:
        # Getting the paramters from API call
        Tenure_value = float(request.args.get('Tenure'))
        WarehouseToHome_value=float(request.args.get('WarehouseToHome'))
        CashbackAmount_value = float(request.args.get('CashbackAmount'))
        NumberOfDeviceRegistered_value = int(request.args.get('NumberOfDeviceRegistered'))
        SatisfactionScore_value = int(request.args.get('SatisfactionScore'))
        NumberOfAddress_value = int(request.args.get('NumberOfAddress'))
        Complain_value = int(request.args.get('Complain'))
        OrderAmountHikeFromlastYear_value = float(request.args.get('OrderAmountHikeFromlastYear'))
        OrderCount_value = float(request.args.get('OrderCount'))
        DaySinceLastOrder_value = float(request.args.get('DaySinceLastOrder'))
                
        # Calling the funtion to get predictions
        prediction_from_api=FunctionGeneratePrediction(
                                                Inp_Tenure= Tenure_value, 
                                                Inp_WarehouseToHome=WarehouseToHome_value, 
                                                Inp_CashbackAmount=CashbackAmount_value, 
                                                Inp_NumberOfDeviceRegistered=NumberOfDeviceRegistered_value, 
                                                Inp_SatisfactionScore=SatisfactionScore_value, 
                                                Inp_NumberOfAddress=NumberOfAddress_value,
                                                Inp_Complain=Complain_value, 
                                                Inp_OrderAmountHikeFromlastYear=OrderAmountHikeFromlastYear_value, 
                                                Inp_OrderCount=OrderCount_value,
                                                Inp_DaySinceLastOrder=DaySinceLastOrder_value
                                                )

        return (prediction_from_api)
    
    except Exception as e:
        return('Something is not right!:'+str(e))

## Starting the API engine



In [19]:
import os
if __name__ =="__main__":
    
    # Hosting the API in localhost
    app.run(host='127.0.0.1', port=8080, threaded=True, debug=True, use_reloader=False)
    # Interrupt kernel to stop the API

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Aug/2021 21:48:45] "GET /prediction_api?Tenure=12&WarehouseToHome=20&CashbackAmount=200&NumberOfDeviceRegistered=4&SatisfactionScore=4&NumberOfAddress=3&Complain=0&OrderAmountHikeFromlastYear=20&OrderCount=4&DaySinceLastOrder=7 HTTP/1.1" 200 -


## Sample URL to call the API


#### Copy and paste below URL in the web browser
http://127.0.0.1:8080/prediction_api?Tenure=12&WarehouseToHome=20&CashbackAmount=200&NumberOfDeviceRegistered=4&SatisfactionScore=4&NumberOfAddress=3&Complain=0&OrderAmountHikeFromlastYear=20&OrderCount=4&DaySinceLastOrder=7